In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
os.chdir('/home/stud/ath/ath_ws/keypoint_dataset_pipeline')

In [23]:
from config import config
from utils import logger

import numpy as np
import h5py
import random
import torch

In [24]:
def print_hdf5_structure(reader):
    def print_group(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
            
    reader._file.visititems(print_group)

In [25]:
class StatsReader:
    def __init__(self):
        filepath = f'{config.paths[config.task.name].output}/data.hdf5'
        self._file = h5py.File(filepath, 'r')

        self.cam = 'cam0'
        assert self.cam in ['cam0', 'cam1']
    
        # [width, height]
        self.original_image_shape = [640, 480]
        self.crop_image_shape = [630, 476]
        
        self._init_groups_read_mode()

    def _init_groups_read_mode(self):
        self._detector = self._file[f'{self.cam}/detector']
        self._matcher = self._file[f'{self.cam}/matcher']
        self._filter = self._file[f'{self.cam}/filter']
        self._matches = self._file[f'{self.cam}/matches']

        self.detector_normalised = self._detector['normalised']
        self.detector_confidences = self._detector['confidences']

        self.matcher_warp = self._matcher['warp'] # only one you will need
        self.matcher_certainty = self._matcher['certainty']

        self.filter_normalised = self._filter['normalised']
        self.filter_confidences = self._filter['confidences']

        self.cropped_image_reference_coords = self._matches['crop/reference_coords']
        self.cropped_image_target_coords = self._matches['crop/target_coords']
    
    def close(self):
        self._file.close()


In [37]:
config.task.frame_filtering = 'normal'

reference_coords_total_count = 0
target_coords_total_count = 0

for track in config.task.tracks:
    config.task.track = track
    logger.info(f'Track : {track}')
    config.task.dataset_kind = track[:2]

    filepath = f'{config.paths[config.task.name].output}/data.hdf5'
    print(filepath)

    reader = StatsReader()
    # print_hdf5_structure(reader)

    for cam in config.task.cams:
        config.task.cam = cam
        logger.info(f'Cam : {cam}')

        refs_from = reader._file[f'{cam}/matches/crop/reference_coords']
        tars_from = reader._file[f'{cam}/matches/crop/target_coords']

        print(f'Number of Reference Coords  : {len(refs_from)}')
        print(f'Number of Target Coords     : {len(tars_from)}')

        reference_coords_total_count += len(refs_from)
        target_coords_total_count += len(tars_from)
    
    reader.close()
    
print(f'Total Reference Coords : {reference_coords_total_count}')
print(f'Total Target Coords : {target_coords_total_count}')

INFO     | <module> | Track : MOO01_hand_puncher_1
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO01_hand_puncher_1/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 610
Number of Target Coords     : 610
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 610
Number of Target Coords     : 610
INFO     | <module> | Track : MOO02_hand_puncher_2


/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO02_hand_puncher_2/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 959
Number of Target Coords     : 959
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 961
Number of Target Coords     : 961
INFO     | <module> | Track : MOO03_hand_shooter_easy
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO03_hand_shooter_easy/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 486
Number of Target Coords     : 486
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 489
Number of Target Coords     : 489
INFO     | <module> | Track : MOO04_hand_shooter_hard
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO04_hand_shooter_hard/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 832
Number of Target Coords     : 832
INFO     | <module> | Cam : cam1
Number of Re